# An analysis of the fractional step method
This is a notebook for understanding and reproducing fractional step method introduced in the ariticle<br> *J.B. Perot, An analysis of the fractional step method, J. Comput. Phys. 108 (1993) 51–58.*

## 1. Introduction
The **fractional step method** has a few major drawbacks. In general, the method is **first-order accurate in time**, and serious confusion and/or disagreement concerning boundary conditions and the details of the methods implementation exists. This paper shows that these problems can be resolved by resisting the urge to view the fractional step method as a time splitting and, instead, viewing it as an approximate block Lu factorization of the fully discretized equations.

Consider the non-dimensionalized Navier-Stokes equations:<br>
$$\frac{\partial \boldsymbol{\boldsymbol{u}}}{\partial t} + \boldsymbol{\boldsymbol{u}} \cdot \nabla \boldsymbol{\boldsymbol{u}} = -\nabla p + \frac{1}{Re} \nabla^2 \boldsymbol{\boldsymbol{u}} \tag{1a}$$
$$\nabla \cdot \boldsymbol{\boldsymbol{u}} = 0 \tag{1b} $$

Straight forward discretization of the equations will produce a system of equations of this term:<br>
$$\begin{pmatrix}
A & G \\
D & 0
\end{pmatrix}
\begin{pmatrix}
\boldsymbol{u}^{n+1} \\
p^{n+1} 
\end{pmatrix}=
\begin{pmatrix}
\boldsymbol{r} \\
0
\end{pmatrix}+
\begin{pmatrix}
bc_{1} \\
bc_{2}
\end{pmatrix} \tag{2}$$

where $A, G$ and $D$ are submatrices, $\boldsymbol{v}^{n+1}$ and $p^{n+1}$ are the unknown discrete velocity and pressure vectors, and the right-hand side vector,$\boldsymbol{r} $, contains all those quantities that are already known.

For example, if the <font color='purple'>**diffusive terms**</font> are updated using the trapezoidal or [Crank-Nicholson method](#crank-nicholson-method), the <font color='orange'>**convective terms**</font> are updated using the [second-order AdamsBashforth method](#adams-bashforth-method), and the spatial discretization is accom-plished with a finite volume discretization or with lumped finite elements,then the Navier-Stokes equations become
$$ \frac{\boldsymbol{u}^{n+1}-\boldsymbol{u}^{n}}{\Delta t}+\textcolor{orange}{\frac{3}{2}H(\boldsymbol{u}^{n})-\frac{1}{2}H(\boldsymbol{u}^{n-1})}=-G\textcolor{red}{p^{n+1}}+\textcolor{purple}{\frac{1}{2Re}L(\boldsymbol{u}^{n+1}+\boldsymbol{u}^{n-1})}+\textcolor{blue}{(bc_{1})}\tag{3a} $$

$$ Du^{n+1}=0+\textcolor{blue}{(bc_{2})}\tag{3b}$$

From See equation (3a)  we can obtain the expression of $A$ and $r$:
$$A=\frac{1}{\Delta t}I-\frac{1}{2Re}L=\frac{1}{\Delta t}(I-\frac{\Delta t}{2Re}L)\tag{4a}$$
$$r=\frac{1}{\Delta t}u^{n}+\frac{1}{2Re}L(u^{n-1})-[\frac{3}{2}H(u^{n})-\frac{1}{2}H(u^{n-1})\tag{4b}]$$

Having committed ourselves to a formulation of the formgiven by Eq.(2), we must note that this system is not easy to solve. Although the system is typically sparse, it is also large and indefinite. The submatrices $D$ and $G$ are not square, and only the submatrix $A$ is invertable. Ideally, we would like to simplify the problem and take advantage of some of the properties of the submatrices that might make inversion easier, For instance, usually $D = G^{T} $, and often $A$ is **symmetric and positive definite**. On Cartesian finite difference or finite volume grids, $A$ is block **diagonal** and can be further factored into a series of tridiagonals.

## 2. Block Decomposition
We can take an approximation of Eq.(2) as follows:
$$\begin{pmatrix}
A & (\Delta t A)G \\
D & 0
\end{pmatrix}
\begin{pmatrix}
\boldsymbol{u}^{n+1} \\
p^{n+1} 
\end{pmatrix}=
\begin{pmatrix}
\boldsymbol{r} \\
0
\end{pmatrix}+
\begin{pmatrix}
bc_{1} \\
bc_{2}
\end{pmatrix} \tag{5}$$
In this case the pressure gradient term in the momentum equations has been altered. For our previous example $(\Delta t A)=[I-(\Delta t/2 Re)L] $, so the relation (5) is a **first-order temporal approximation** of Eq.(2) with an error term
$(\Delta t/2 Re) LGp^{n+1}$. Fortunately, <font color='red'>the error term is diffusive and tends to aid stability</font>. 

<font color='red' >**The approximate system given by (5) can be factor into the form of LU decomposition**</font>
$$\begin{pmatrix}
A & 0 \\
D & -\Delta tDG \\
\end{pmatrix}
\begin{pmatrix}
\boldsymbol{u}^{*} \\
p^{n+1}\\
\end{pmatrix}=
\begin{pmatrix}
\boldsymbol{r} \\
0\\
\end{pmatrix}+
\begin{pmatrix}
bc_{1} \\
bc_{2}\\
\end{pmatrix} \tag{6a}$$
$$\begin{pmatrix}
I & \Delta t G \\
0 & I\\
\end{pmatrix}
\begin{pmatrix}
\boldsymbol{u}^{n+1} \\
p^{n+1}\\
\end{pmatrix}=
\begin{pmatrix}
\boldsymbol{u}^{*} \\
p^{n+1}\\
\end{pmatrix} \tag{6b}$$

where $u^{*}$ is an intermediate velocity vector. The matrix $A$ is invertible, and the matrix $D$ is square and invertible. The matrix $DG$ is also square and invertible, and the matrix $D$ is invertible.

# Crank-Nicholson method

**Crank-Nicholson method** or  **Crank-Nicholson scheme** use the <font color='red'>average of the values at the current time step </font>$n$ <font color='red'>and the next time step </font>$n+1$ to evaluate the time derivative. It is a **second-order method** in time, **implicit** in time, and is numerically stable. It is named after John Crank and Phyllis Nicolson. Next, we use the first-order wave equation $ \frac{\partial \boldsymbol{\boldsymbol{u}}}{\partial t}+a\frac{\partial \boldsymbol{\boldsymbol{u}}}{\partial x} =0 $ as a demonstration. the discretized equation is:
$$ \frac{u_{j}^{n+1}-u_{j}^{n}}{\Delta t} + \frac{a}{2} \frac{u_{j+1}^{n}-u_{j-1}^{n}}{2\Delta x}+ \frac{a}{2} \frac{u_{j+1}^{n+1}-u_{j-1}^{n+1}}{2\Delta x}=0$$

Its correlation equation is:
$$ u_{t}+au_{x}=-\frac{a\Delta x^{2}}{12}(12+c^{2})u_{xxx}-\frac{a\Delta x^{4}}{240}(2+10c^{2}+3c^{4})u_{xxxxx}-\dots$$
And its gain is:
$$ G=\frac{1-icsin(\beta)/2}{1+icsin(\beta)/2}$$

Obviously, |G| = 1, and the truncation error only contains terms of odd derivatives. The scheme is **dispersive**. In practical calculations, artificial viscosity needs to be added to smooth out the non-physical oscillations near the discontinuities in the numerical solutions.

# Adams-Bashforth method

**Adams-Bashforth method** or **Adams-Bashforth scheme** is a commonly used three-layer explicit scheme,which means using 3 time steps. Still take the first-order wave equation $ \frac{\partial \boldsymbol{\boldsymbol{u}}}{\partial t}+a\frac{\partial \boldsymbol{\boldsymbol{u}}}{\partial x} =0 $ for demonstration, we have the discretized equation:
$$ \frac{u_{j}^{n+1}-u_{j}^{n}}{\Delta t} + a(\frac{3}{2} \frac{u_{j+1}^{n}-u_{j-1}^{n}}{2\Delta x}- \frac{1}{2} \frac{u_{j+1}^{n-1}-u_{j-1}^{n-1}}{2\Delta x})=0$$
or
$$ u_{j}^{n+1}=u_{j}^{n}-\frac{c}{4}[3(u_{j+1}^{n}-u_{j-1}^{n})-(u_{j+1}^{n-1}-u_{j-1}^{n-1})]$$ 
And its gain satisfies the following equation:
$$ G^{2}=G(1-\frac{3}{2}icsin\beta)+\frac{1}{2}csin\beta$$
$$G^{\pm}=\frac{1}{4}(2-3isin\beta+\Delta^{\pm}),\Delta^{\pm}=4-4icsin\beta-9c^{2}sin\beta \equiv r^{2}e^{2i\theta}$$
Thus,
$$|4G^{\pm}|^{2}=4+r^{2}+9c^{2}\mp\sqrt{2}(2\sqrt{r^{2}+4-9c^{2}}+3c\sqrt{r^{2}-4+9c^{2}})$$
We can obtain that $G^{-}>1$

But this rate of growth can be relatively easily suppressed by other physical dissipation or artificial numerical dissipation. Therefore, **when solving problems with dissipation, the Adams–Bashforth scheme is often used to discretize the advection terms**